In [ ]:
import json
import os
import pprint
import time

import ee
import pandas as pd

from IPython.display import Image, display
import ipyplot


In [ ]:
ee.Initialize(
    project='ee-cmorton',
    opt_url='https://earthengine-highvolume.googleapis.com'
)

In [ ]:
# The Ocean mask is True for water, so flip it for updateMask call so that land pixels are 1
land_mask = ee.Image('projects/openet/assets/features/water_mask').Not()
# Apply the NLCD/NALCMS water mask (anywhere it is water, set the ocean mask 
land_mask = land_mask.where(ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS").unmask(18).eq(18), 0)
# land_mask = land_mask.And(ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS").unmask(18).neq(18))

wrs2_list = sorted(
    ee.FeatureCollection('projects/openet/assets/features/wrs2/custom')
    .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
    .aggregate_histogram('wrs2_tile').keys().getInfo(),
    reverse=True
)
# wrs2_list = wrs2_list
print(len(wrs2_list))
# print(wrs2_list)

wrs2_skip_list = []

image_size = 700

# 0 - white, 1 - no fill (green), 2 - shadow (dark blue), 3 - snow (light blue), 4 - cloud (light gray), 5 - water (purple), 6 - ocean mask
fmask_palette = "ffffff, 9effa1, blue, 00aff2, dddddd, purple, bfbfbf"
fmask_max = 6


In [ ]:
refl_sr_bands = ['SR_RED', 'SR_GREEN', 'SR_BLUE', 'QA_PIXEL', 'QA_RADSAT']
refl_sr_bands_dict = ee.Dictionary({
    'LT04': ['SR_B3', 'SR_B2', 'SR_B1', 'QA_PIXEL', 'QA_RADSAT'],
    'LT05': ['SR_B3', 'SR_B2', 'SR_B1', 'QA_PIXEL', 'QA_RADSAT'],
    'LE07': ['SR_B3', 'SR_B2', 'SR_B1', 'QA_PIXEL', 'QA_RADSAT'],
    'LC08': ['SR_B4', 'SR_B3', 'SR_B2', 'QA_PIXEL', 'QA_RADSAT'],
    'LC09': ['SR_B4', 'SR_B3', 'SR_B2', 'QA_PIXEL', 'QA_RADSAT'],
})
refl_toa_bands_dict = ee.Dictionary({
    'LT04': ['B3', 'B2', 'B1'],
    'LT05': ['B3', 'B2', 'B1'],
    'LE07': ['B3', 'B2', 'B1'],
    'LC08': ['B4', 'B3', 'B2'],
    'LC09': ['B4', 'B3', 'B2'],
})
# sr_coll_dict = ee.Dictionary({
#     'LT05': ee.ImageCollection('LANDSAT/LT05/C02/T1_L2'),
#     'LE07': ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'),
#     'LC08': ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'),
#     'LC09': ee.ImageCollection('LANDSAT/LC09/C02/T1_L2'),
# })
# toa_coll_dict = ee.Dictionary({
#     'LT05': ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA'),
#     'LE07': ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA'),
#     'LC08': ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA'),
#     'LC09': ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA'),
# })

rgb_bands = {
    'LT04': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LT05': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LE07': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LC08': ['SR_B4', 'SR_B3', 'SR_B2'],
    'LC09': ['SR_B4', 'SR_B3', 'SR_B2'],
}


In [ ]:
def image_stats(landsat_img):
    scene_id = ee.String(landsat_img.get('scene_id'))
    landsat_type = scene_id.slice(0, 4)

    # Note, we can't rename the TOA bands here since the simple cloud score is expecting a raw TOA image
    landsat_toa_img = ee.Image(landsat_img.get('landsat_toa_img'))

    default_stats = ee.Dictionary({
        'SCENE_ID': scene_id,
        'UNMASKED_PIXELS': -1, 'TOTAL_PIXELS': -1,
        'CLOUD_PIXELS': -1, 'CIRRUS_PIXELS': -1, 'DILATE_PIXELS': -1, 
        'SHADOW_PIXELS': -1, 'SNOW_PIXELS': -1, 'WATER_PIXELS': -1,
        'SR_RED': -1, 'SR_GREEN': -1, 'SR_BLUE': -1,
        'UNMASKED_SR_RED': -1, 'UNMASKED_SR_GREEN': -1, 'UNMASKED_SR_BLUE': -1,
        'TOA_RED': -1, 'TOA_GREEN': -1, 'TOA_BLUE': -1,
        'UNMASKED_TOA_RED': -1, 'UNMASKED_TOA_GREEN': -1, 'UNMASKED_TOA_BLUE': -1,
        'CLOUD_COVER_LAND': landsat_img.get('CLOUD_COVER_LAND'), 
        'MORAN_1K': -1, 'MORAN_2K': -1, 'MORAN_4K': -1, 'MORAN_8K': -1,
        #'SSEBOP_ETF_count': -9999, 'SSEBOP_ETF_mean': -9999, 
    })

    # Get the cloud mask (including the snow mask for now)
    qa_img = ee.Image(landsat_img.select(['QA_PIXEL']))
    cloud_mask = qa_img.rightShift(3).bitwiseAnd(1).neq(0)
    fmask_mask = qa_img.rightShift(3).bitwiseAnd(1).neq(0)
    
    # if cirrus_flag:
    cirrus_mask = qa_img.rightShift(2).bitwiseAnd(1).neq(0).And(fmask_mask.Not())
    fmask_mask = fmask_mask.Or(cirrus_mask)
    
    # if dilate_flag:
    dilate_mask = qa_img.rightShift(1).bitwiseAnd(1).neq(0).And(fmask_mask.Not())
    fmask_mask = fmask_mask.Or(dilate_mask)

    # if shadow_flag:
    shadow_mask = qa_img.rightShift(4).bitwiseAnd(1).neq(0).And(fmask_mask.Not())
    fmask_mask = fmask_mask.Or(shadow_mask)
    
    # if snow_flag:
    snow_mask = qa_img.rightShift(5).bitwiseAnd(1).neq(0).And(fmask_mask.Not())
    fmask_mask = fmask_mask.Or(snow_mask)

    # if water_flag:
    water_mask = qa_img.rightShift(7).bitwiseAnd(1).neq(0).And(fmask_mask.Not())

    # CGM - This isn't working correctly, don't apply
    # # # Apply a small erosion/dilation
    # # fmask_mask = (
    # #     fmask_mask
    # #     .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.circle(radius=1, units='pixels'))
    # #     .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.circle(radius=2, units='pixels'))
    # #     # .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.circle(radius=30, units='meters'))
    # #     # .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.circle(radius=60, units='meters'))
    # #     # .reproject(qa_img.projection())
    # # )

    # Saturated mask (only keep unmasked saturated pixels)
    # Flag as saturated if any of the RGB bands are saturated
    #   or change .gt(0) to .gt(7) to flag if all RGB bands are saturated
    # Comment out rightShift line to flag if saturated in any band
    bitshift = ee.Dictionary({'LT04': 0, 'LT05': 0, 'LE07': 0, 'LC08': 1, 'LC09': 1});
    saturated_mask = (
        landsat_img.select('QA_RADSAT')
        .rightShift(ee.Number(bitshift.get(landsat_type))).bitwiseAnd(7)
        .gt(0)
    )
    saturated_mask = saturated_mask.where(fmask_mask, 0)

    # Simple cloud score (ACCA)
    # Only keep unmasked ACCA pixels
    acca_mask = ee.Algorithms.Landsat.simpleCloudScore(landsat_toa_img).select(['cloud']).gte(100)
    acca_mask = acca_mask.where(fmask_mask, 0)

    # Flip to set cloudy pixels to 0 and clear to 1
    fmask_update_mask = fmask_mask.Not()

    rr_mean_params = {
        'reducer': ee.Reducer.mean().unweighted(),
        'geometry': qa_img.geometry(), 
        'crs': qa_img.projection().crs(), 
        'crsTransform': [30, 0, 15, 0, -30, 15],
        'bestEffort': False,
        'maxPixels': 1E12,
    }
    rr_count_params = {
        'reducer': ee.Reducer.count().unweighted(),
        'geometry': qa_img.geometry(), 
        'crs': qa_img.projection().crs(), 
        'crsTransform': [30, 0, 15, 0, -30, 15],
        'bestEffort': False,
        'maxPixels': 1E12,
    }
    
    tile_scale = 1
    if tile_scale != 1:
        rr_mean_params['tileScale'] = tile_scale
        rr_count_params['tileScale'] = tile_scale

    refl_sr_nomask_bands = (
        landsat_img.select(['SR_RED', 'SR_GREEN', 'SR_BLUE'])
        .multiply([0.0000275]).add([-0.2]).clamp(0, 1)
    )
    refl_sr_masked_bands = (
        landsat_img.select(
            ['SR_RED', 'SR_GREEN', 'SR_BLUE'], 
            ['UNMASKED_SR_RED', 'UNMASKED_SR_GREEN', 'UNMASKED_SR_BLUE']
        )
        .multiply([0.0000275]).add([-0.2]).clamp(0, 1)
        .updateMask(fmask_update_mask)
    )
    refl_toa_nomask_bands = (
        landsat_toa_img.select(
            refl_toa_bands_dict.get(landsat_type), 
            ['TOA_RED', 'TOA_GREEN', 'TOA_BLUE']
        )
    )
    refl_toa_masked_bands = (
        landsat_toa_img.select(
            refl_toa_bands_dict.get(landsat_type), 
            ['UNMASKED_TOA_RED', 'UNMASKED_TOA_GREEN', 'UNMASKED_TOA_BLUE']
        )
        .updateMask(fmask_update_mask)
    )
    refl_mean_stats = (
        refl_sr_nomask_bands
        .addBands(refl_sr_masked_bands)
        .addBands(refl_toa_nomask_bands)
        .addBands(refl_toa_masked_bands)
        .updateMask(land_mask)
        .reduceRegion(**rr_mean_params)
    )

    # Compute the masked count stats (these may be the same, not sure yet)
    # If they are, then it may make more sense to compute the masked and unmasked count
    count_stats = (
        landsat_img.select(['SR_RED'], ['UNMASKED_PIXELS']).updateMask(fmask_update_mask)
        .addBands([
            landsat_img.select(['SR_RED'], ['TOTAL_PIXELS']),
            cloud_mask.selfMask().rename(['CLOUD_PIXELS']),
            cirrus_mask.selfMask().rename(['CIRRUS_PIXELS']),
            dilate_mask.selfMask().rename(['DILATE_PIXELS']),
            shadow_mask.selfMask().rename(['SHADOW_PIXELS']),
            snow_mask.selfMask().rename(['SNOW_PIXELS']),
            water_mask.selfMask().rename(['WATER_PIXELS']),
            saturated_mask.selfMask().rename(['SATURATED_PIXELS']),
            acca_mask.selfMask().rename(['ACCA_PIXELS']),
        ])
        .updateMask(land_mask)
        .reduceRegion(**rr_count_params)
    )

    # # Compute the SSEBop ETf stats
    # ssebop_stats = (
    #     ee.Image(landsat_img).select('SSEBOP_ETF').divide(10000)
    #     .updateMask(land_mask)
    #     .reduceRegion(
    #         reducer=ee.Reducer.mean().unweighted()
    #             .combine(ee.Reducer.count().unweighted(), '', True)
    #             # .setOutputs(['SSEBOP_ETF_MEAN', 'SSEBOP_ETF_COUNT'])
    #         ,
    #         geometry=qa_img.geometry(), 
    #         crs=qa_img.projection().crs(), 
    #         crsTransform=[30, 0, 15, 0, -30, 15],
    #         bestEffort=False,
    #         maxPixels=1E10,
    #     )
    # )

    # # Compute the grid stats for the default cloud mask
    # mask_img = fmask_mask.updateMask(land_mask)
    # # image_url = (
    # #     grid_mask_img.visualize(min=0, max=1, palette=['orange', 'blue'])
    # #     .getThumbURL({'region': mask_img.geometry().bounds(1, 'EPSG:4326'), 'dimensions': 1024})
    # # )
    # # ipyplot.plot_images([image_url], img_width=1024)

    # # First build the covering grid
    # # 30, 60, 120, 240, 480, 960, 1920, 3840, 7680, 15360, 30720, 61140, 122880
    # grid_coll = covering_grid(qa_img, 7680)
    # # grid_coll = qa_img.geometry().coveringGrid(qa_img.projection().crs(), 7680)
    # # grid_coll = qa_img.geometry().coveringGrid(qa_img.projection().crs(), 15360)

    # # Compute the scene percent cloud
    # # We may already have this value in this function but recompute for now
    # image_masked_pct = ee.Number(mask_img.rename('masked_pct').reduceRegion(**rr_mean_params).get('masked_pct'))
    # #print(image_cloud_pct.getInfo())
    
    # # Then compute the stats for each grid cell
    # def grid_stats_func(ftr):
    #     stats = mask_img.addBands(mask_img.mask()).rename(['masked', 'total']).reduceRegion(**{
    #         'reducer': ee.Reducer.mean().unweighted(), 'geometry': ftr.geometry(), 
    #         'crs': qa_img.projection().crs(), 'crsTransform': [30, 0, 15, 0, -30, 15], 
    #         'bestEffort': False, 'maxPixels': 1E12,
    #     })

    #     stats = stats.set('masked_dec', ee.Number(stats.get('masked')).multiply(10).round().divide(10))
        
    #     # Start with a dictionary set to 0 to handle the grid cells that don't intersect the image
    #     stats = ee.Dictionary({'masked': 0, 'total': 0}).combine(stats, True)
        
    #     # Compute the difference in cloudiness from the scene average
    #     #stats = stats.set('diff', ee.Number(stats.get('masked')).subtract(image_masked_pct))
        
    #     return ee.Feature(ftr.geometry(), stats)

    # # Only keep grid cells that cover the unmasked portion of the scene by some amount
    # min_total = 0.5
    # grid_coll = ee.FeatureCollection(grid_coll.map(grid_stats_func)).filter(ee.Filter.gt('total', min_total))
    # grid_stats = grid_coll.aggregate_stats('diff')
    # grid_histogram
    # grid_stats = ee.Dictionary({
    #     'GRID_COUNT': grid_stats.get('total_count'),
    #     'GRID_MASK_PCT': image_masked_pct,
    #     # 'GRID_COUNT': grid_stats.get('valid_count'),
    #     # 'GRID_DIFF_MEAN': grid_stats.get('mean'),
    #     # 'GRID_DIFF_MIN': grid_stats.get('min'),
    #     # 'GRID_DIFF_MAX': grid_stats.get('max'),
    #     # 'GRID_DIFF_SD': grid_stats.get('total_sd'),
    #     # 'GRID_DIFF_VAR': grid_stats.get('total_var'),
    #     # 'GRID_DIFF_SUMSQ': grid_stats.get('sum_sq'),
    # })

    output_stats = (
        default_stats
        .combine(refl_mean_stats, overwrite=True)
        .combine(count_stats, overwrite=True)
    )
    
    return ee.Feature(None, output_stats)


# image_id = 'LANDSAT/LC08/C02/T1_L2/LC08_019037_20200911'  # random
# # image_id = 'LANDSAT/LC09/C02/T1_L2/LC09_023034_20220703'  # clustered
# # image_id = 'LANDSAT/LC09/C02/T1_L2/LC09_029027_20240328'  # clustered
# test_img = (
#     ee.Image(image_id)
#     .select(refl_sr_bands_dict.get('LE07'), ['SR_RED', 'SR_GREEN', 'SR_BLUE', 'QA_PIXEL', 'QA_RADSAT'])
#     .set('scene_id', image_id.split('/')[-1])
#     .set('landsat_toa_img', ee.Image(image_id.replace('T1_L2', 'T1_TOA')))
# )
# output = image_stats(test_img)
# pprint.pprint(output.getInfo())


In [ ]:
def fmask(landsat_img):
    # Add the fmask image on top of the true color image
    qa_img = landsat_img.select('QA_PIXEL')
    fill_mask = qa_img.bitwiseAnd(1).neq(0)                  # bits: 0
    dilate_mask = qa_img.rightShift(1).bitwiseAnd(1).neq(0)  # bits: 1
    cirrus_mask = qa_img.rightShift(2).bitwiseAnd(1).neq(0)  # bits: 2
    cloud_mask = qa_img.rightShift(3).bitwiseAnd(1).neq(0)   # bits: 3
    shadow_mask = qa_img.rightShift(4).bitwiseAnd(1).neq(0)  # bits: 4
    snow_mask = qa_img.rightShift(5).bitwiseAnd(1).neq(0)    # bits: 5
    clear_mask = qa_img.rightShift(6).bitwiseAnd(1).neq(0)   # bits: 6
    water_mask = qa_img.rightShift(7).bitwiseAnd(1).neq(0)   # bits: 7
    # cloud_conf = qa_img.rightShift(8).bitwiseAnd(3)          # bits: 8, 9
    # shadow_conf = qa_img.rightShift(10).bitwiseAnd(3)        # bits: 10, 11
    # snow_conf = qa_img.rightShift(12).bitwiseAnd(3)          # bits: 12, 13
    # cirrus_conf = qa_img.rightShift(14).bitwiseAnd(3)        # bits: 14, 15

    # Saturated pixels
    # Flag as saturated if any of the RGB bands are saturated
    #   or change .gt(0) to .gt(7) to flag if all RGB bands are saturated
    # Comment out rightShift line to flag if saturated in any band
    bitshift = ee.Dictionary({'LANDSAT_4': 0, 'LANDSAT_5': 0, 'LANDSAT_7': 0, 'LANDSAT_8': 1, 'LANDSAT_9': 1});
    saturated_mask = (
        landsat_img.select('QA_RADSAT')
        .rightShift(ee.Number(bitshift.get(ee.String(landsat_img.get('SPACECRAFT_ID'))))).bitwiseAnd(7)
        .gt(0)
    )
    
    # Old "Fmask" style image
    fmask_img = (
        qa_img.multiply(0)
        .where(landsat_img.select(['SR_B4']).mask().eq(0), 1)
        # .where(saturated_mask, 6)
        .where(water_mask, 5)
        .where(shadow_mask, 2)
        .where(snow_mask, 3)
        .where(cloud_mask.Or(dilate_mask).Or(cirrus_mask), 4)
        # .add(shadow_mask.multiply(2))
        # .add(snow_mask.multiply(3))
        # .add(cloud_mask.Or(dilate_mask).Or(cirrus_mask).multiply(4))
        # .add(cloud_mask.Or(dilate_mask).multiply(4))
        # .add(cloud_mask.And(cloud_conf).multiply(4))
        # .add(water_mask.multiply(5))
    )
    
    return fmask_img.updateMask(fmask_img.neq(0)).rename(['fmask'])


In [ ]:
# Read in the scene skip list
scene_skip_url = '../v2p1.csv'
# scene_skip_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1.csv'
scene_skip_df = pd.read_csv(scene_skip_url)
scene_skip_list = list(scene_skip_df['SCENE_ID'].values)
print(f'Skip list images: {len(scene_skip_list)}')

scene_cloudscore_url = '../v2p1_cloudscore.csv'
# scene_cloudscore_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1_cloudscore.csv'
scene_cloudscore_list = list(pd.read_csv(scene_cloudscore_url)['SCENE_ID'].values)
print(f'Skip cloudscore images: {len(scene_cloudscore_list)}')


In [ ]:
# Compute stats and save CSV to bucket
start_year = 2024
end_year = 2025
years = list(range(start_year, end_year+1))

count_threshold_pct_min = 99
count_threshold_pct_max = 101

month = 1

new_skip_scenes = []
new_skip_count = 0
           
for year in years: 
    print(f'{year}')

    # Process a single month for now
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    l4_sr_coll = (
        ee.ImageCollection('LANDSAT/LT04/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 71)
        .filter(ee.Filter.inList('system:index', scene_skip_list).Not())
        .select(refl_sr_bands_dict.get('LT04'), refl_sr_bands)
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l5_sr_coll = (
        ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 71)
        .filter(ee.Filter.inList('system:index', scene_skip_list).Not())
        .select(refl_sr_bands_dict.get('LT05'), refl_sr_bands)
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l7_sr_coll = (
        ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 71)
        .filter(ee.Filter.inList('system:index', scene_skip_list).Not())
        .select(refl_sr_bands_dict.get('LE07'), refl_sr_bands)
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l8_sr_coll = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 71)
        .filter(ee.Filter.inList('system:index', scene_skip_list).Not())
        .select(refl_sr_bands_dict.get('LC08'), refl_sr_bands)
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l9_sr_coll = (
        ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 71)
        .filter(ee.Filter.inList('system:index', scene_skip_list).Not())
        .select(refl_sr_bands_dict.get('LC09'), refl_sr_bands)
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    if year < 1993:
        landsat_sr_coll = l5_sr_coll.merge(l4_sr_coll)
    elif year in range(1993, 1999):
        landsat_sr_coll = l5_sr_coll
    elif year in range(1999, 2013):
        landsat_sr_coll = l5_sr_coll.merge(l7_sr_coll)
    elif year in range(2013, 2023):
        landsat_sr_coll = l8_sr_coll.merge(l7_sr_coll)
    elif year >= 2023:
        landsat_sr_coll = l9_sr_coll.merge(l8_sr_coll)

    if landsat_sr_coll.size().getInfo() == 0:
        print('  No landsat sr images in year/tile')
        continue

    l4_toa_coll = (
        ee.ImageCollection('LANDSAT/LT04/C02/T1_TOA')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
        # .map(lambda img: img.set('image_id', img.get('system:id')))
    )
    l5_toa_coll = (
        ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l7_toa_coll = (
        ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l8_toa_coll = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )
    l9_toa_coll = (
        ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA')
        .filterDate(start_date, end_date)
        .filterBounds(ee.Geometry.BBox(-124, 44, -67, 52))
        .map(lambda img: img.set({'scene_id': img.get('system:index')}))
    )

    if year < 1993:
        landsat_toa_coll = l5_toa_coll.merge(l4_toa_coll)
    elif year in range(1993, 1999):
        landsat_toa_coll = l5_toa_coll
    elif year in range(1999, 2013):
        landsat_toa_coll = l5_toa_coll.merge(l7_toa_coll)
    elif year in range(2013, 2023):
        landsat_toa_coll = l8_toa_coll.merge(l7_toa_coll)
    elif year >= 2023:
        landsat_toa_coll = l9_toa_coll.merge(l8_toa_coll)

    landsat_coll = ee.Join.saveFirst(matchKey='landsat_toa_img').apply(
        landsat_sr_coll, landsat_toa_coll, 
        ee.Filter.equals(leftField='scene_id', rightField='scene_id'),
    )     
    scene_id_list = landsat_coll.aggregate_array('scene_id').getInfo()
    print(f'  Images: {len(scene_id_list)}')


    for scene_id in scene_id_list:
        print(scene_id)

        landsat_type = scene_id.split('_')[0].upper()

        # Build the joined image for the image stats function
        landsat_sr_coll = (
            ee.ImageCollection([ee.Image(f'LANDSAT/{landsat_type}/C02/T1_L2/{scene_id.upper()}')])
            .select(refl_sr_bands_dict.get(landsat_type), refl_sr_bands)
            .map(lambda img: img.set({'scene_id': img.get('system:index')}))
        )
        landsat_toa_coll = (
            ee.ImageCollection([ee.Image(f'LANDSAT/{landsat_type}/C02/T1_TOA/{scene_id.upper()}')])
            .map(lambda img: img.set({'scene_id': img.get('system:index')}))
        )
        landsat_coll = ee.Join.saveFirst(matchKey='landsat_toa_img').apply(
            landsat_sr_coll, landsat_toa_coll, 
            ee.Filter.equals(leftField='scene_id', rightField='scene_id'),
        )     
        output_info = ee.ImageCollection(landsat_coll).map(image_stats).getInfo()

        # Build a dataframe of the stats
        stats_df = pd.DataFrame([ftr['properties'] for ftr in output_info['features']])
        stats_df['DATE'] = stats_df['SCENE_ID'].str.slice(12, 20)
        stats_df['WRS2'] = 'p' + stats_df['SCENE_ID'].str.slice(5, 8) + 'r' + stats_df['SCENE_ID'].str.slice(8, 11)

        # Compute the ratios
        # stats_df['ACCA_COUNT_RATIO'] = stats_df['ACCA_PIXELS'] / stats_df['TOTAL_PIXELS']
        stats_df['SNOW_COUNT_RATIO'] = stats_df['SNOW_PIXELS'] / stats_df['TOTAL_PIXELS']
        # stats_df['SHADOW_COUNT_RATIO'] = stats_df['SHADOW_PIXELS'] / stats_df['TOTAL_PIXELS']
        # stats_df['WATER_COUNT_RATIO'] = stats_df['WATER_PIXELS'] / stats_df['TOTAL_PIXELS']
        stats_df['MASKED_PIXELS'] = (
            stats_df['CLOUD_PIXELS'] + stats_df['CIRRUS_PIXELS'] + stats_df['DILATE_PIXELS']
            + stats_df['SHADOW_PIXELS']
            + stats_df['SNOW_PIXELS']
            + stats_df['WATER_PIXELS']
            + stats_df['ACCA_PIXELS']
            # + stats_df['SATURATED_PIXELS']
        )
        stats_df['CLOUD_COUNT_RATIO'] = stats_df['MASKED_PIXELS'] / stats_df['TOTAL_PIXELS']
        # stats_df['CLOUD_COUNT_RATIO'] = stats_df['UNMASKED_PIXELS'] / stats_df['TOTAL_PIXELS']

        # Filter on the overall cloud count ratio
        stats_df = stats_df[stats_df['CLOUD_COUNT_RATIO'] < (count_threshold_pct_max / 100)]
        stats_df = stats_df[stats_df['CLOUD_COUNT_RATIO'] >= (count_threshold_pct_min / 100)]

        if stats_df.empty:
            continue

        # Show the image if it is cloudy
        landsat_img = ee.Image(f'LANDSAT/{landsat_type}/C02/T1_L2/{scene_id}')
        landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
        landsat_sr_img = landsat_img.select(rgb_bands[landsat_type]).multiply([0.0000275]).add([-0.2])

        # Landsat true color image
        landsat_url = (
            landsat_sr_img.where(land_mask.unmask().eq(0), 0.25)
            .getThumbURL({'min': 0.0, 'max': 0.30, 'gamma': 1.25, 'region': landsat_region, 'dimensions': image_size})
        )
    
        # Landsat true color with Fmask
        fmask_url = (
            landsat_sr_img.where(land_mask.unmask().eq(0), 0.25).visualize(min=0, max=0.3, gamma=1.25)
            .blend(fmask(landsat_img).where(land_mask.unmask().eq(0), fmask_max).visualize(bands='fmask', min=0, max=fmask_max, palette=fmask_palette))
            .getThumbURL({'region': landsat_region, 'dimensions': image_size})
        )
    
        print('#'*80)
        print(
            f'  {scene_id}  {row["TOTAL_PIXELS"]:>10d}  {row["UNMASKED_PIXELS"]:>10d}'
            f'  ({row["CLOUD_COUNT_RATIO"]:>0.2f}) ({row["SNOW_COUNT_RATIO"]:>0.2f}) {row["CLOUD_COVER_LAND"]}'
        )
        ipyplot.plot_images([landsat_url, fmask_url], img_width=image_size)

        new_skip_scenes.append(scene_id)
        new_skip_count += 1

print('\nNew Scenes')
if new_skip_scenes:
    for scene_id in new_skip_scenes:
        print(scene_id)

print('\nDone')
